In [5]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import logging
import requests
import json
import pandas as pd

In [6]:
# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)

# Retrieve your API keys from environment variables
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [8]:
URL = "https://storage.googleapis.com/generativeai-downloads/data/State_of_the_Union_Address_30_January_1961.mp3"

In [11]:
!wget -q $URL -O sample.mp3

zsh:1: command not found: wget


In [ ]:
your_file = genai.upload_file(path='sample.mp3')